In [156]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch.nn as nn
from __future__ import print_function
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [157]:
housing=pd.read_csv('Housing.csv')
housing.head(5)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [158]:
housing.mainroad = housing.mainroad.map(dict(yes=1,no=0))
housing.guestroom = housing.guestroom.map(dict(yes=1,no=0))
housing.basement = housing.basement.map(dict(yes=1,no=0))
housing.hotwaterheating = housing.hotwaterheating.map(dict(yes=1,no=0))
housing.airconditioning = housing.airconditioning.map(dict(yes=1,no=0))
housing.prefarea = housing.prefarea.map(dict(yes=1,no=0))

housing.head(5)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [159]:
housing.shape

(545, 13)

In [160]:
num_vars=['price', 'area','bathrooms','bedrooms', 'stories', 'parking']
data_set=housing[num_vars]
data_set.head()

,price,area,bathrooms,bedrooms,stories,parking
0,13300000,7420,2,4,3,2
1,12250000,8960,4,4,4,3
2,12250000,9960,2,3,2,2
3,12215000,7500,2,4,2,3
4,11410000,7420,1,4,2,2


In [161]:
data_set.shape

(545, 6)

In [162]:
scaler = MinMaxScaler()
data_set[num_vars]= scaler.fit_transform(data_set[num_vars])
data_set.head()

C:\Users\nrashvan\AppData\Local\Temp\1\ipykernel_15120\1593678094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set[num_vars]= scaler.fit_transform(data_set[num_vars])


,price,area,bathrooms,bedrooms,stories,parking
0,1.000000,0.396564,0.333333,0.6,0.666667,0.666667
1,0.909091,0.502405,1.000000,0.6,1.000000,1.000000
2,0.909091,0.571134,0.333333,0.4,0.333333,0.666667
3,0.906061,0.402062,0.333333,0.6,0.333333,1.000000
4,0.836364,0.396564,0.000000,0.6,0.333333,0.666667


In [163]:
X = data_set.iloc[:, 1:6].values
X.shape


(545, 5)

In [164]:
Y = data_set.iloc[:, 0].values
Y.shape

(545,)

In [165]:
Y =Y.reshape(-1,1)
Y.shape

(545, 1)

In [166]:
t_u=torch.tensor(X)
t_c=torch.tensor(Y)
t_c.shape

torch.Size([545, 1])

In [167]:
t_u.shape

torch.Size([545, 5])

In [168]:
n_samples= t_u.shape[0]
n_val= int(0.2 * n_samples)

shuffled_indices= torch.randperm(n_samples)

train_indices= shuffled_indices[:-n_val]
val_indices= shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 50, 315, 325, 416,  95, 397, 293, 349, 445, 194,  74, 517,  80, 512,
         214, 314,  70, 202, 340, 252, 188, 492, 199, 177, 429, 289,  47, 102,
          97, 229, 225, 257,  48,   3, 259, 525, 527, 292, 204, 472, 379, 489,
         403, 374, 454, 398, 270, 157, 271, 460, 514, 218,  84, 134, 466, 420,
         196,  34, 285,  86, 419, 337, 328, 453, 250, 544,  57, 368, 329, 377,
         413, 160, 333, 296, 476, 185, 467, 311,  49, 529, 440, 100, 166, 523,
         140, 117, 258,  42, 459, 418, 146, 187, 104, 471, 540, 162, 247, 541,
         111, 430, 273, 354, 145, 496, 347,   8, 493,  91,  38, 536,  22, 361,
          28, 385, 255, 246,  60, 186, 530,  56, 244, 112, 176, 308, 399, 406,
         330, 490,  32, 230, 276, 520, 226, 407, 138, 228, 515, 542, 422, 241,
         510, 508,  11,  13, 462, 390, 487, 132, 363,  18, 394, 178, 414, 148,
         306,  66, 172, 261,  92, 498, 268, 121, 404, 372, 243, 505, 383,  61,
         428, 358, 163, 303, 316, 334, 310, 483, 366

In [169]:
train_t_u= t_u[train_indices]
train_t_c=t_c[train_indices]

val_t_u= t_u[val_indices]
val_t_c= t_c[val_indices]


In [170]:
from collections import OrderedDict

In [211]:
seq_model=nn.Sequential(OrderedDict([
    ('hidden_linear1', nn.Linear(5, 18)),
    ('hidden_activation1', nn.Tanh()), 
    ('hidden_linear2', nn.Linear(18,6)),
    ('hidden_activation2', nn.Tanh()),
    ('output_linear', nn.Linear(6, 1))
    
]))

seq_model

Sequential(
  (hidden_linear1): Linear(in_features=5, out_features=18, bias=True)
  (hidden_activation1): Tanh()
  (hidden_linear2): Linear(in_features=18, out_features=6, bias=True)
  (hidden_activation2): Tanh()
  (output_linear): Linear(in_features=6, out_features=1, bias=True)
)

In [212]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

hidden_linear1.weight torch.Size([18, 5])
hidden_linear1.bias torch.Size([18])
hidden_linear2.weight torch.Size([6, 18])
hidden_linear2.bias torch.Size([6])
output_linear.weight torch.Size([1, 6])
output_linear.bias torch.Size([1])


In [213]:
seq_model.output_linear.bias

Parameter containing:
tensor([-0.3149], requires_grad=True)

In [214]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1,n_epochs +1):
        
        t_p_train=model(train_t_u)
        loss_train= loss_fn(t_p_train, train_t_c)
        
        t_p_val=model(val_t_u)
        loss_val= loss_fn(t_p_val, val_t_c)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        if epoch ==1 or epoch % 20==0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f}, "
                  f" Validation loss {loss_val.item():.4f}")

In [215]:
train_t_u = train_t_u.float()

In [216]:
print(train_t_u.dtype)

torch.float32


In [217]:
val_t_u=val_t_u.float()
print(val_t_u.dtype)

torch.float32


In [218]:
train_t_c=train_t_c.float()
print(train_t_c.dtype)

torch.float32


In [219]:
val_t_c=val_t_c.float()
print(val_t_c.dtype)

torch.float32


In [224]:
optimizer= optim.SGD(seq_model.parameters(), lr=1e-1)
training_loop(
    n_epochs=200,
    optimizer= optimizer,
    model=seq_model,
    loss_fn=nn.MSELoss(),
   train_t_u=train_t_u,
   val_t_u= val_t_u,
     train_t_c= train_t_c,
   val_t_c= val_t_c)
print('output', seq_model(val_t_u))
print('answer',  val_t_c)
print('hidden', seq_model.hidden_linear1.weight.grad)
print('hidden', seq_model.hidden_linear2.weight.grad)
print('hidden', seq_model.output_linear.weight.grad)

Epoch 1, Training loss 0.0123,  Validation loss 0.0145
Epoch 20, Training loss 0.0122,  Validation loss 0.0144
Epoch 40, Training loss 0.0121,  Validation loss 0.0143
Epoch 60, Training loss 0.0120,  Validation loss 0.0142
Epoch 80, Training loss 0.0119,  Validation loss 0.0141
Epoch 100, Training loss 0.0118,  Validation loss 0.0140
Epoch 120, Training loss 0.0117,  Validation loss 0.0139
Epoch 140, Training loss 0.0117,  Validation loss 0.0139
Epoch 160, Training loss 0.0116,  Validation loss 0.0138
Epoch 180, Training loss 0.0116,  Validation loss 0.0138
Epoch 200, Training loss 0.0115,  Validation loss 0.0137
output tensor([[0.3744],
        [0.1018],
        [0.2954],
        [0.0993],
        [0.3223],
        [0.1412],
        [0.2201],
        [0.2907],
        [0.2962],
        [0.3000],
        [0.1519],
        [0.1028],
        [0.2873],
        [0.2819],
        [0.3220],
        [0.1356],
        [0.1757],
        [0.4353],
        [0.3060],
        [0.2266],
        [0.4